In [ ]:
!pip install transformers datasets accelerate scikit-learn torch

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)

In [ ]:
train_df = pd.read_csv("/content/sample_data/train.csv")
test_df = pd.read_csv("/content/sample_data/test.csv")
train_df = train_df.rename(columns={"target": "label"})
test_df = test_df.rename(columns={"target": "label"})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
train_val_split = train_dataset.train_test_split(test_size=0.2)
train_split = train_val_split["train"]
val_split = train_val_split["test"]

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_tokenized = train_split.map(tokenize_function, batched=True)
val_tokenized = val_split.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)
train_tokenized = train_tokenized.remove_columns(["text", "keyword", "location", "id"])
val_tokenized = val_tokenized.remove_columns(["text", "keyword", "location", "id"])
test_tokenized = test_tokenized.remove_columns(["text", "keyword", "location", "id"])

In [ ]:
print(train_tokenized[0].keys())

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions)
    }

training_args = TrainingArguments(
    output_dir="./distilbert_distress",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",  # disable wandb
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics
)

In [ ]:
print("Starting training...")
trainer.train()

In [ ]:
print("Predicting on test set...")
predictions = trainer.predict(test_tokenized)
pred_labels = predictions.predictions.argmax(axis=1)

In [ ]:
for i in range(10):
    print(f"Tweet: {test_df.iloc[i]['text']}")
    print(f"Prediction: {'Distressed' if pred_labels[i] == 1 else 'Not Distressed'}\n")

In [ ]:
trainer.save_model("./my_distress_detector")
tokenizer.save_pretrained("./my_distress_detector")

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("./my_distress_detector")
tokenizer = DistilBertTokenizer.from_pretrained("./my_distress_detector")